# CIS600 - Social Media & Data Mining
###  
<img src="https://www.syracuse.edu/wp-content/themes/g6-carbon/img/syracuse-university-seal.svg?ver=6.3.9" style="width: 200px;"/>

# $n$-grams & Word2Vec 

###  March 29, 2018

#### Today's material comes mainly from *Foundations of Statistical Natural Language Processing* (*Manning & Schütze*) and *`Word2Vec` Explained* (*Goldberg & Levy*).

## Statistical Inference

### In the broadest sense, *statistical inference* consists of taking data from an unknown distribution and making inferences based on the data about that distribution.

### A famous example would be *hypothesis testing*, e.g. "do these two population means really differ?"

### Let's take as motivation the *language modeling* task. This is to predict the next word in a sequence from the preceding word. This might seem like a weird motivation, but it leads to good things. It is sometimes called a *Shannon game*. Have you heard of [Claude Shannon](https://en.wikipedia.org/wiki/Claude_Shannon)?

## $n$-gram

### Recall $n$-grams. The $n$-gram modeling task is to estimate the probability

## $P(w_n \mid w_1,\ldots,w_{n-1})$.

### The preceding $n-1$ words are together called the *history* of the next word $w_n$. The history will not be the entire document leading up to $w_n$, for this would make the parameter space much too large. Generally, we work with $n$-grams for $n = 2,3,4$.

### A model predicting the next word from the preceding $n-1$ is called a $(n-1)^{th}$ order Markov model or simply an $n$-gram model.

### There is a tradeoff between reliability and discrimination. If we have more features, then we can discriminate better. On the other hand, we might then *bin* the data into such small subsets that we cannot learn what's in the bins very well. Here is an example in terms of the length of history used in an $n$-gram model:

> Sue swallowed the large green _

### Likely next words are *pill*, *bean* or even *frog*. But consider just the fragment

> the large green _

### where likely next words include also *mountain*, *car* or *tree*. We would like to be able to use enough parameters (i.e. a long enough history) to rule out these words in a sentence like the first.

### Assuming a vocabulary of $20000$ words, we can see the combinatorial explosion associated with increasing values of $n$.

- $1^{st}$ order (bigram): $20,000 \times 19,999 = 40,000,000$
- $2^{nd}$ order (trigram): $20,000^2 \times 19,999 = 8,000,000,000,000$
- $3^{th}$ order (four-gram): $20,000^3 \times 19,999 = 1.6 \times 10^{17}$

### So let's stick with bigrams. How do we build bigram models? Remember, we're modeling the probabilities

## $P(w_n \mid w_1,\ldots,w_{n-1})$.

### A first step is the formula

## $P(w_n \mid w_1,\ldots,w_{n-1}) = \frac{P(w_1,\ldots,w_n)}{P(w_1,\ldots,w_{n-1})}$

### Suppose the corpus contains $N$ words, and the vocabulary has $V$ words. Then there are $V^n$ different $n$-gram probabilities to be estimated (many probabilities could be $0$) and $V$ possible values for the target (last word to be predicted). Generally, let $B$ denote this number of different values whose probabilities we're trying to predict.

### Finally, $C(w_1,\ldots,w_n)$ denotes the frequency (count) of the $n$-gram $w_1,\ldots,w_n$ in the training text.

### Rmk: we'll use a built-in corpus in a second here, and the `bigram` function, but here is an easily overlooked tip from *Manning & Schütze*

> If we append $n-1$ dummy start symbols to the beginning of the text, we can then also say that the corpus consists of $N$ $n$-grams, with a uniform amount of conditioning available for the next word in all cases.

## Maximum Likelihood Estimator

### This is basically this most naïve thing you could do and it doesn't work very well, but let's look at it anyway.

## $P_{MLE}(w_1,\ldots,w_n) = \frac{C(w_1,\ldots,w_n)}{N}$

## and

## $P_{MLE}(w_n \mid w_1,\ldots,w_{n-1}) = \frac{C(w_1,\ldots,w_n)}{C(w_1,\ldots,w_{n-1})}$

### To illustrate, consider the following *trigram* example. Suppose we're looking at instances where the two preceding words are *comes across*, $10$ of them in total, of which $8$ are followed by *as*, another is followed by *a* and the last is followed by *more*. To sum up: 

- *comes across as*: $\times$ $8$
- *comes across more*: $\times$ $1$
- *comes across a*: $\times$ $1$

### This would give probability estimates $P(as) = 0.8$, $P(more) = 0.1$ and so on.

### *The main problem with this approach is sparseness*. In particular, the longer the $n$-gram (i.e. larger $n$), the harder it is to find multiple instances of a single $n$-gram. For example, from above

> swallowed the large green

### versus

> the large green

### Let's see this in action. We'll use Jane Austen's *Persuasion*, from the Gutenberg corpus collection. First, here is a tabular summary from *Manning & Schütze*

<img src="notebook-images/ngram.png" style="width: 800px;"/>

In [3]:
import nltk
from nltk.corpus import gutenberg
gutenberg.fileids()[:4]

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt']

### Let's use *Emma* and *Sense & Sensibility* for training, then *Persuasion* for testing.

In [2]:
# Getting just lowercase words
import string

def simplify_doc(d):
    return [w.lower() for w in d if w not in string.punctuation]

In [5]:
# Getting documents
persuasion = gutenberg.words('austen-persuasion.txt')
emma = gutenberg.words('austen-emma.txt')
sense = gutenberg.words('austen-sense.txt')

# Processing with our function
persuasion, emma, sense = simplify_doc(persuasion), simplify_doc(emma), simplify_doc(sense)

In [9]:
# Combining my training corpus into one list
emma_sense = emma + sense

In [11]:
bigrams = [x for x in nltk.ngrams(emma_sense,2)]
trigrams = [x for x in nltk.ngrams(emma_sense,3)]
quadrigrams = [x for x in nltk.ngrams(emma_sense,4)]

### First we can look at the frequency distribution

In [12]:
es_freq = nltk.FreqDist(emma_sense)

In [13]:
es_freq.most_common(10)

[('to', 9355),
 ('the', 9306),
 ('and', 8387),
 ('of', 7863),
 ('a', 5221),
 ('i', 5182),
 ('her', 5020),
 ('it', 4285),
 ('was', 4259),
 ('in', 4167)]

### Note that we get a different ranking, but similar estimates to what you see in the table.

In [17]:
dict(es_freq)['she'] / len(emma_sense)

0.01361291522318569

In [18]:
es_freq.most_common(20)

[('to', 9355),
 ('the', 9306),
 ('and', 8387),
 ('of', 7863),
 ('a', 5221),
 ('i', 5182),
 ('her', 5020),
 ('it', 4285),
 ('was', 4259),
 ('in', 4167),
 ('she', 3953),
 ('not', 3388),
 ('be', 3280),
 ('that', 3191),
 ('you', 3171),
 ('he', 2914),
 ('as', 2657),
 ('had', 2622),
 ('for', 2609),
 ('but', 2327)]

### The sentence we're evaluating is

> In person, she was inferior to both sisters, and had, even in her bloom, only reached the dignity of being “a fine girl."

### Let's look at our $n$-grams and compare with the table. For example, we can look in the first column, second row...

In [32]:
cond_person = [v for (w,v) in bigrams if w=='person']
person_freq = nltk.FreqDist(cond_person)
person_dist = dict(person_freq)
for key in person_dist:
    person_dist[key] = person_dist[key] / len(cond_person)
print(person_dist)

{'and': 0.10218978102189781, 'to': 0.072992700729927, 'had': 0.014598540145985401, 'grows': 0.0072992700729927005, 'fixed': 0.0072992700729927005, 'much': 0.0072992700729927005, 'able': 0.0072992700729927005, 'than': 0.0072992700729927005, ',"': 0.014598540145985401, 'whose': 0.029197080291970802, 'if': 0.0072992700729927005, 'can': 0.014598540145985401, 'at': 0.029197080291970802, 'i': 0.043795620437956206, 'you': 0.0072992700729927005, 'must': 0.0072992700729927005, 'or': 0.029197080291970802, 'of': 0.014598540145985401, 'good': 0.0072992700729927005, 'she': 0.014598540145985401, '--': 0.014598540145985401, 'well': 0.0072992700729927005, 'who': 0.10218978102189781, 'elegant': 0.0072992700729927005, '."': 0.0364963503649635, 'among': 0.0072992700729927005, 's': 0.043795620437956206, 'has': 0.0072992700729927005, 'will': 0.0072992700729927005, 'for': 0.014598540145985401, 'was': 0.021897810218978103, 'whom': 0.0364963503649635, 'there': 0.0072992700729927005, 'like': 0.0072992700729927

In [33]:
person_dist['she']

0.014598540145985401

### Hm, so it looks like the conditional probability is actually higher in this case (than in the table).

### We shouldn't be able to find (in, person, `word`) in our trigrams...

In [30]:
in_person = [z for (w,v,z) in trigrams if (w,v)==('in','person')]
in_person

['or', 'and', '--', 'had', '."', 'and', 'mrs', 'was']

### Perhaps it's the capitalization...let's try *was inferior*

In [31]:
in_person = [z for (w,v,z) in trigrams if (w,v)==('was','inferior')]
in_person

[]

### That makes sense.

### Generally, we need lots more data before $n$-gram models like this become useful.

## Adjustments: Laplace & Lidstone

### Two alternative estimators, which can be derived through Bayesian reasoning, are the *Laplace* and *Lidstone* estimators. 

## $P_{Lap}(w_1, \ldots,w_n) = \frac{C(w_1,\ldots,w_n) + 1}{N + B}$

### This has the advantage that unseen $n$-grams are still given a positive weight. But there is actually too much weight given to them in aggregate:

> Although each previously unseen bigram has been given a very low probability, because there are so many of them, 99.97% of the probability mass has actually been given to unseen bigrams...the empirical model finds that only 9.2% of the bigrams in further text were previously unseen.

### The Lidstone estimator tries to address this issue

## $P_{Lid}(w_1, \ldots,w_n) = \frac{C(w_1,\ldots,w_n) + \lambda}{N + \lambda B}$

### Taking $\mu = N / (N + \lambda B)$ lets us write

## $P_{Lid}(w_1,\ldots,w_n) =\mu \frac{C(w_1,\ldots,w_n)}{N} + (1-\mu)\frac{1}{B}$

### which shows that $P_{Lid}$ is a convex combination of the MLE and a *uniform prior* (another Bayesian term).

## Word2Vec

### The idea that carries over from the above discussion is that we are modeling probabilities of words based on surrounding *context* words. In *Word2Vec*, the problem is posed in such a way that the resulting distribution estimates actually *capture some of the meaning of words represented*.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/ab/Ludwig_Wittgenstein.jpg/410px-Ludwig_Wittgenstein.jpg" style="width: 400px;"/>

### In particular, dot products of the resulting vector representations give a measure of similarity in meaning. Furthermore, rather famously, vector arithmetic can even express *analogies*:

### $King - Man + Woman = Queen$

### This doesn't always work, obviously, and we are not discovering some mathematical basis for the meaning of words, but it is an important embedding. So let's learn where it comes from.

## Skip-gram

### We don't actually need the full notion of *skip-gram*, but here is a demonstration. But we will be interested in call *context words* surrounding a given *center word*.

In [ ]:
from nltk.util import skipgrams
sent = "Insurgents killed in ongoing fighting".split()
list(skipgrams(sent, 2, 2))

In [ ]:
list(skipgrams(sent,3,2))

## An Optimization Problem

### Everything is an optimization problem. Here is the set up - we are essentially maximizing the likelihood of our corpus subject to the calculated parameters (the values in the word vectors).

### Our objective function, per *Goldberg-Levy*:

## $J(\theta) = \large\prod_{w \in Text}\big[\large\prod_{c \in C(w)}p(c \mid w;\theta)\big]$

### *THIS IS THE THING WE WANT TO MAXIMIZE*

### Let us calmly break this down into pieces and understand what is going on. First, the $Text$ is our whole corpus. A $w$ is a word (or appropriate term) in our corpus $Text$. With a given word $w$ fixed, the *context* $C(w)$ of $w$ is all the words $c$ in a window around $w$. The width of the windows you want to use is a hyperparameter of this model; it's not included in $\theta$. The variable $\theta$ contains all of the vector representations of the words in the vocabulary.

### We can expand that a bit to make it a little more precise, and using slightly different notation. Below, words of  a corpus are indexed $1, \ldots, T$.

## $J(\theta) = \large\prod_{1 \leq t \leq T}\big[\large\prod_{-m \leq j \leq m,\text{ } j \neq 0}p(w_{t+j} \mid w_t;\theta)\big]$

### That's not much better, but says explicitly what these contexts look like.

### Right now, this is not much of a model - *what is this value* $p(c \mid w; \theta)$ ? Here we make a pretty big leap and choose the *softmax* function to model these probabilities:

## $(x_1,\ldots,x_n) \mapsto \big(\frac{e^{x_1}}{\sum_{i=1}^n e^{x_i}}, \ldots, \frac{e^{x_n}}{\sum_{i=1}^n e^{x_i}}\big)$

### That's a general definition. In this case, we're looking at

# $p(c \mid w; \theta) = \frac{e^{v_c\cdot v_w}}{\sum_{c' \in C}e^{v_{c'}\cdot v_w}}$

### You might see this expressed in other (better) ways. In particular, separate letters $u$ and $v$ can be used to distinguish the context word vector representations from the center word vector representations. Here that distinction is made via separate index sets ($c$ runs through context words, and $w$ through the vocabulary of the training set). Goldberg-Levy have a useful remark on this point:

> we assume that the words and the contexts come from distinct
vocabularies, so that, for example, the vector associated with the word dog will be different
from the vector associated with the context dog. This assumption follows the literature, where
it is not motivated. One motivation for making this assumption is the following: consider the
case where both the word dog and the context dog share the same vector v. Words hardly
appear in the contexts of themselves, and so the model should assign a low probability to
p(dog|dog), which entails assigning a low value to v · v which is impossible.

### Moving on: we now have something concrete (mathematically speaking)! We know exactly the form of the objective function, but how to optimize? It turns out to be very expensive. Here is one smart method people figured out...

### (Rmk: numerically, you would take a logarithm at this point. Big products are numerically unstable, and the logarithm is monotonic, so that optima do not change after that transformation).

<img src="https://upload.wikimedia.org/wikipedia/commons/8/81/Logarithm_plots.png" style="width: 400px;"/>

## Negative Sampling

### Let's derive a different objective function. This is an alternative to the objective defined above but produces similar embeddings.

### Start with a word-context pair $(w,c)$. We'll model the probability that it comes from our actual dataset, $D$. Denote this by $p(D=1 \mid (w,c))$.

### In this setup, our goal is to find word representations so that we maximize the probability that our observations come from the data. In equations, we are maximizing

## $\large\prod_{(w,c) \in D}p(D=1 \mid w,c;\theta)\large\prod_{(w,c) \in D'}p(D=0 \mid w,c;\theta)$

### Here, $D'$ are some *negative samples*, often chosen at random, known not to belong to the dataset.

## Implementations

### `Word2Vec` is implemented in the `gensim` python package and in `tensorflow`, and in other places.